# XAS data validation by a simple machine learning model

-----

**Objective**: Train a computer to recognize when a measured spectrum looks like X-ray Absorption Spectroscopy (XAS) data.

-----

At my beamline, [BMM](https://www.bnl.gov/nsls2/beamlines/beamline.php?r=6-BM), we try to run 24 hours per day by relying upon robust instrumentation and well-tested automation. We have ways of lining up 10s of hours of data collection and letting the beamline run itself. From time to time, however, something goes wrong.  Maybe a detector fails ... maybe a sample has falls off the sample holder ... maybe the user tells the automation to do the wrong thing ... maybe gremlins overrun the beamline.  Who knows?

Here is what we expect XAS data to look like:

In [ ]:
from IPython.display import Image
Image(filename='./static/ok_data.png', width=300)

But, sometimes something goes awray and we see garbage like this:

In [ ]:
Image(filename='./static/bad_data.png', width=300)

We need a simple sort of data evaluation.  My talk today is not about XAS data reduction ... nor anlaysis ... nor interpretation ....

My plan is to show a way to flag a spectrum **as its measurement completes** as being either
1. "these data are probably OK" or
2. "someone's attention is probably needed"?

My starting point is the basic observation is that this sort of identification problem is fundementally the same at the famous Iris Classification Problem &ndash; which is the "Hello World!" of machine learning.

## The Iris Classification Problem

In this classic problem, we work with a data set of observations of the morphology of the flowers of three species of iris:

![irises](./static/irises.png)
[(image source)](https://data-flair.training/blogs/iris-flower-classification/)

**Sepal**: One of the usually green leaflike structures composing the outermost part of a flower. Sepals often enclose and protect the bud and may remain after the fruit form.

**Petal**: One of the often brightly colored parts of a flower immediately surrounding the reproductive organs.

Note that the shapes of the petals and sepals of these three species are different from one another.

In [ ]:
import pandas
import sklearn.datasets
iris_set = sklearn.datasets.load_iris()

i = pandas.DataFrame()
i['sepal length'] = iris_set.data[:,0]
i['sepal width']  = iris_set.data[:,1]
i['petal length'] = iris_set.data[:,2]
i['petal width']  = iris_set.data[:,3]
i['target']       = iris_set.target
i['species']      = iris_set.target_names[iris_set.target]
i

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

formatter = plt.FuncFormatter(lambda i, *args: iris_set.target_names[int(i)])

plt.scatter(i['petal length'], i['petal width'], c=i['target'])
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.xlabel('petal length')
plt.ylabel('petal width')
plt.title("Classification: Petal measurements")
## plotting credit: http://stephanie-w.github.io/brainscribble/classification-algorithms-on-iris-dataset.html

In [ ]:
plt.scatter(i['sepal length'], i['sepal width'], c=i['target'])
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.xlabel('sepal length')
plt.ylabel('sepal width')
plt.title("Classification: Sepal measurements")

In [ ]:
plt.scatter(i['petal width'], i['sepal length'], c=i['target'])
plt.colorbar(ticks=[0, 1, 2], format=formatter)
plt.xlabel('petal width')
plt.ylabel('sepal length')
plt.title("Classification: PW vs. SL")

Thanks to nicely contrasting colors and the human brain's penchant for finding patterns, you can look at these representations of the iris dataset and see that the species cluster according to the dimensions of their sepals and petals.

Remember that these pictures are 2-dimensional samplings of a 4-dimensional space of sepal and petal measurements.  There are 6 such 2D samplings.  The actual clustering in this data set should be considered on a 4D manifold.

Now, imagine picking a new iris of unknown species with the intent of identifying it.  You might measure the length and width of its sepal and petal and drop the new measurement onto that 4D manifold.  The idea is to determine its species on the basis of the nearby, tagged data points.

To implement this in numbers (as opposed to human perception), we'll use an algorithm called "K Nearest Neighbors" (KNN).  To explain KNN, I'll simply plagiarize Wikipedia:

![KNN](./static/KnnClassification.svg)
[(image source)](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)

Quoth Wikipedia: "The test sample (green dot) should be classified either to blue squares or to red triangles. If k = 3 (solid line circle) it is assigned to the red triangles because there are 2 triangles and only 1 square inside the inner circle. If k = 5 (dashed line circle) it is assigned to the blue squares (3 squares vs. 2 triangles inside the outer circle)."

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(i[['sepal length', 'sepal width', 'petal length', 'petal width']], i['target'], random_state=0)
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

Here is the documentation from scikit-learn on the iris dataset:
https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html and for more information, follow the other links cited above.

So ... how does a fun botany problem relate to XAS?

## Classifying XAS spectra

Let's start by importing modules, fetching a catalog of data from Tiled, and defining some variables and utility functions.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
import numpy
import os

## create BMM's tiled catalog
from tiled.client import from_uri
c = from_uri("https://tiled-demo.blueskyproject.io/api")
cat=c['bmm']['raw']

## Import the UIDs and human-assigned scores for the training corpus from files in ./data/
with open('./data/training_set') as infile:
    uids = infile.read().split('\n')
uids_of_training_set = [x for x in uids if (len(x) > 1)]

import json, itertools
with open("./data/tags.json") as infile:
    tags = json.load(infile)

## a couple of utility functions for munging UID strings    
def full_uid(short):
    for u in uids_of_training_set:
        if u[-8:] == short:
            return u
    return None    
def short_uid(full):
    return full[-8:]

## import from CSV files
def fetch_xas_scan_from_csv(uid):
    fname = uid + '.csv'
    data = pandas.read_csv(os.path.join("data", "ML_corpus", fname))
    data.plot("dcm_energy", "xmu", xlabel='Energy (eV)', ylabel='$\mu$(E)')

## import from tiled
def fetch_xas_scan(uid, type='fluorescence', plot=True):
    if len(uid) < 10:
        uid = full_uid(uid)
    if uid is None:
        print("not a valid UID")
        return
    data = cat[uid].primary.read()
    
    ## apply knowledge of BMM databroker records to contruct mu(E) from xarray columns
    if type == 'transmission':                             # tranmission data
        data['xmu'] = numpy.log(data['I0']/data['It'])
    elif 'xs' in cat[uid].metadata['start']['detectors']:  # fluorescence with Xspress3
        elem = cat[uid].metadata['start']['XDI']['Element']['symbol']
        data['xmu'] = (data[elem+'1']+data[elem+'2']+data[elem+'3']+data[elem+'4'])/data['I0']
    else:                                                  # fluorescence, analog readout, prior to Xspress3
        if cat[uid].metadata['start']['XDI']['Element']['symbol'] == 'Fe':
            data['xmu'] = (data['DTC1']+data['DTC2']+data['DTC3']+data['DTC4'])/data['I0']
        elif cat[uid].metadata['start']['XDI']['Element']['symbol'] == 'Ti':
            data['xmu'] = (data['DTC2_1']+data['DTC2_2']+data['DTC2_3']+data['DTC2_4'])/data['I0']
        elif cat[uid].metadata['start']['XDI']['Element']['symbol'] == 'Ce':
            data['xmu'] = (data['DTC3_1']+data['DTC3_2']+data['DTC3_3']+data['DTC3_4'])/data['I0']
            
    if plot is True:
        plt.xlabel('Energy (eV)')
        plt.gca().set_ylabel('$\mu$(E)')
        plt.plot(data['dcm_energy'], data['xmu'])
    return(data)


In [ ]:
len(uids_of_training_set)

Most of the data from this presentation is taken from a weekend in mid-2021 -- during time-of-covid, when all the experiments at BMM were mail-in, and at a moment when I was monitoring the beamline from home.  Being a nice weekend day, I set many hours of data collection running and then walked away.

### Data, good and bad

That weekend, I was working on ceramic samples from colleagues at the University of Sheffield. We were measuring XAS on the iron, cerium, and titanium edges.

Here are some examples of reasonable data:

In [ ]:
## a good one (Fe)
this=fetch_xas_scan('4de69926')

In [ ]:
## a good one (Ce)
this=fetch_xas_scan('4f3c2372')

In [ ]:
# a good one (Ti)
this=fetch_xas_scan('6916040c')

At some point during the weekend, something ... **BAD** ... happened. In truth, I don't quite remember the exact details, but here's the gist.  At the time, we were using a analog electronics to read the pulse trains from the detector and send those signals into VME scalar. Something in that signal chain got into a bad state such that the pulses were not being read cleanly.

This went on for something like 10 hours, measuring garbage, before I finally noticed.

Here are a couple of examples of **BAD** data:

In [ ]:
## a bad one
this = fetch_xas_scan('88b9e311')

In [ ]:
## another bad one
this = fetch_xas_scan('64887ce3')

### Preparing the training data

This is a "supervised learning" problem.  That means that a human (me!) goes through the training data and tags each spectrum as *good* or *bad*.  

The data, taken from BMM's DataBroker, were measured between July 9 and July 13 in 2020. These data are made available by the tiled demo server, like so:

```python
from tiled.client import from_uri
c = from_uri("https://tiled-demo.blueskyproject.io/api")
cat=c['bmm']['raw']
```

The variable `cat` contains a catalog of DataBroker entries measured at BMM.  For those familiar with `bluesky`, the records in `cat` are much the same as what you would get by querying DataBroker directly from the `bluesky` command line, like so:

```python
measurement = db[uid]
```

The UIDs for the training data are listed in [this file](./data/training_set).

I wrote a simple shell script that stepped through each spectrum in the training set, displayed a plot of each spectrum, and prompted me for a score for each spectrum.

**"good data"**: score = 1, i.e. a spectrum that looked to my human eye like it stepped up then wiggled.

**"bad data"**: score = 0, i.e. a spectrum that looked to my human eye like it **did not** step up then wiggle.

(Side note: human tagging of a data set is tedious and error prone.  An ideal model would be tolerant of errors in the training set.)

In [ ]:
n=8
print(f'There are {len(uids_of_training_set)} UIDs in the training set.  Here are the first {n}:\n')
uids_of_training_set[:8]

The scoring of the test (i.e. 1/0 for good/bad) was saved as [a JSON file](./data/ML_corpus/tags.json). Let's see what the first eight entries in that JSON file look like:

In [ ]:
dict(itertools.islice(tags.items(),  1, n))

Let's do a spot check on one of the good ones (`04fed2c6` == `'aedf4b8d-b6cb-4939-8e06-5ef704fed2c6'`) and on one of the bad ones (`0920716b` == `'f2acb042-033e-40b3-a4ac-a4d70920716b'`):

In [ ]:
this=fetch_xas_scan('04fed2c6') # this one is tagged as "good"

In [ ]:
this=fetch_xas_scan('0920716b') # this one is tagged as "bad"

Great!  Now we can start constructing our training set.

First thing: we need to "rationalize" the data. The classifier expects all the data to be the same size -- for example, each observation of an iris had 4 data points (width and length of sepal and petal).  Similarly, the XAS spectra in our training set need to have the same number of energy points. Because different scans mght have different numbers of energy point, we will interpolate all the data onto a 401-point grid which is evenly spaced across the energy range of the original XAS scan.

In [ ]:
import numpy
def rationalize_mu(en, mu):
    '''Return energy and mu as a Pandas dataframe with data interpolated onto 
    a "rationalized" grid of equally spaced points.  GRIDSIZE = 401
    '''
    GRIDSIZE = 401
    ee=list(numpy.arange(en[0], en[-1], (en[-1]-en[0])/GRIDSIZE))
    mm=numpy.interp(ee, en, mu)
    if len(ee) > GRIDSIZE:
        ee = ee[:-1]
        mm = mm[:-1]
    df = pandas.DataFrame()
    df['dcm_energy'] = ee
    df['xmu'] = mm
    return(df)

def plot_rationalized_data(data, rat):
    '''Make a quick-n-dirty plot of the original data overplotted by the data
    interpolated onto a 401-point grid.
    '''
    plt.xlabel('Energy (eV)')
    plt.gca().set_ylabel('$\mu$(E)')
    plt.plot(data["dcm_energy"], data["xmu"], label='original')
    plt.legend()
    ax = plt.gca()
    rat.plot("dcm_energy", "xmu", xlabel='Energy (eV)', ylabel='$\mu$(E)', label='rationalized', ax=ax)

data = fetch_xas_scan('04fed2c6', plot=False)
data_rational = rationalize_mu(data['dcm_energy'], data['xmu'])
plot_rationalized_data(data, data_rational)

And here's a bad one:

In [ ]:
data = fetch_xas_scan('0920716b', plot=False)
data_rational = rationalize_mu(data['dcm_energy'], data['xmu'])
plot_rationalized_data(data, data_rational)

### Training and testing the model

Almost ready!  Now, we need to import the entire tagged learning corpus into a form ready to be consumed by the sklearn classifier.

In [ ]:
#%%timeit -n 1 -r 1

from simple_progress_bar import update_progress
corpus = []
scores = []

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

for i,f in enumerate(uids_of_training_set):
    try:
        data = fetch_xas_scan(f, plot=False)
        data_rational = rationalize_mu(data['dcm_energy'], data['xmu'])
        if short_uid(f) in tags:
            corpus.append(list(data_rational['xmu']))
            scores.append(tags[short_uid(f)])
    except:
        pass
    update_progress(i / len(uids_of_training_set))
update_progress(1)

In [ ]:
len(corpus), len(scores)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
clf=KNeighborsClassifier(n_neighbors=1)
X_train, X_test, y_train, y_test = train_test_split(corpus, scores, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

97% success on the training set!  Not bad for an extremely naive approach to the problem.  Let's see if we can't improve upon this without having to do too much more work.

[SciKit Learn](https://scikit-learn.org/stable/index.html) comes with a rather enormous number of
[supervised learning models](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning). Let's try another one ... [a random forest classifier](https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-tree).

This model is based on "decision trees".  In short, the model asks a sequence of abstract questions about the data &ndash; essentially "does this data point look like it's from good data or bad data?"  It makes some number of such decision trees and has them all vote on whether an unknown should be categorized as good data or bad.  (See [here](https://www.ibm.com/cloud/learn/random-foresthttps://www.ibm.com/cloud/learn/random-forest) for a much better explanation.)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

Complete success?!?  Woot!  Now we're gettin' somewhere!

We are already starting to push up against my dilettante's knowledge of machine learning. A more informed choice of classifier can be made (and was, by Phil, in the paper a group of us here at NSLS-II just got published), but let's plow ahead using our random forest.

To get a sense of how this works, let's look at the first item in the test portion of the training set.  We'll check how I tagged it, see what it looks like when plotted, and see how it evaluates using the model:

In [ ]:
y_test[0]

In [ ]:
plt.plot(X_test[0])

In [ ]:
clf.predict([X_test[0]])

The `predict` function returns a 1 or a 0 on the basis of its evaluation of the supplied test data.  In this case, the model and I agree about these data. Yay!

Let's try it on a spectrum not in the training set! Here's an Fe edge spectrum measured on a completely different sample from a completely different area of science which was measured at BMM over a year later:

In [ ]:
unknown = fetch_xas_scan('89353743-7c9a-4d4b-8f2a-34673e69a361', plot=False)
unknown_rational = rationalize_mu(unknown['dcm_energy'], unknown['xmu'])
plot_rationalized_data(unknown, unknown_rational)
#unknown

In [ ]:
clf.predict([list(unknown_rational['xmu'])])

Splendid!  A visual inspection tells us that the new spectrum looks like XAS data and our model agrees!

## Using our model

Once our model is created, we can follow sklearn's hints about [model persistence](https://scikit-learn.org/stable/modules/model_persistence.html).  The model gets serialized to a [joblib](https://joblib.readthedocs.io/en/latest/persistence.html) file.  The file containing the model serialization is part of the [bluesky profile at BMM](https://github.com/NSLS-II-BMM/profile_collection).  Thus this machine learning model is always available and ready to be integrated into our operations.

In practice, we compare *every* spectrum measured against our model.  The plan we use to measure an XAS spectrum includes a loop over the numbr of repetitions reqeusted by the user.  As part of that loop, the data that just finished are rationalized as discussed above and scored by the model.

At BMM, we use Slack to provide feedback to users and staff during the experiment.  In the screenshot below, you can see the result of the data evaluation for each of two repetitions on that sample.  At the end of the two repetitions, the data are merged and lightly process, then a picture of the data are posted to Slack.

![Slack+ML](./static/slack+ml.png)

In this way, user and staff are given a hint about whether the experiment is progressing generally well or not.

## Improving the model

In practice, the model developed in this tutorial is not strong enough for general use.  Here's an example:

In [ ]:
failure = fetch_xas_scan('10cefec0-8239-458d-9cb8-3f8aa371ce79', plot=False)
failure_rational = rationalize_mu(failure['dcm_energy'], failure['xmu'])
plot_rationalized_data(failure, failure_rational)

In [ ]:
clf.predict([list(failure_rational['xmu'])])

Poop!

Those As edge data are obviously excellent data, but the model in its current state does not recognize that.

Over time, I have tagged more data and added them to the model.  The model in use at BMM is still not perfect. False negatives are, by far, the more common failure. Just last week, I instrumented BMM's profile to log the UID of every scan that fails the data evaluation so that I can correctly tag the spectrum and add it to the training set.  Hopefully, over time, the model will become robust against all actual XAS data measured at BMM.

Reliablility in the high 90s still means that every day, a user will ask me "Why did the data evaluation fail? What's wrong with my data?"  Sigh....

The actual implementation of this machine learning model at BMM is contained in [this file](https://github.com/NSLS-II-BMM/profile_collection/blob/master/startup/BMM/ml.py) from [BMM's profile](https://github.com/NSLS-II-BMM/profile_collection). It's use upon conclusion of an XAFS scan and the posting of the result to Slack is shown [here](https://github.com/NSLS-II-BMM/profile_collection/blob/master/startup/BMM/xafs.py#L986-L995https://github.com/NSLS-II-BMM/profile_collection/blob/master/startup/BMM/xafs.py#L986-L995).

-----

As a closing comment, I want to advertise a little project of mine.

Bluesky by Example (https://nsls-ii-bmm.github.io/bluesky-by-example/) is Bluesky documentation by and for beamline staff.  Maybe you'll find something helpful there.  

Or..... Maybe you have something helpful *to contribute*.  Please fork the repo and make a PR to add your own example.  This presentation will eventually become a chapter.

![BBE](./static/bluesky-by-example.png)

